# Kmeans Clustering using Spark.ml
Python 및 sklearn을 사용하여 K-means 클러스터링을 수행

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataFrame").config("spark.sql.repl.eagerEval.enabled", True).getOrCreate()
spark

In [4]:
from sklearn.datasets import load_iris
import pandas as pd
from pyspark.ml.classification import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

iris = load_iris()
df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [5]:
df_iris['Species'] = iris.target
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
df_iris = spark.createDataFrame(df_iris)
df_iris.show(5)

+-----------------+----------------+-----------------+----------------+-------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|Species|
+-----------------+----------------+-----------------+----------------+-------+
|              5.1|             3.5|              1.4|             0.2|      0|
|              4.9|             3.0|              1.4|             0.2|      0|
|              4.7|             3.2|              1.3|             0.2|      0|
|              4.6|             3.1|              1.5|             0.2|      0|
|              5.0|             3.6|              1.4|             0.2|      0|
+-----------------+----------------+-----------------+----------------+-------+
only showing top 5 rows



In [7]:
df_iris.count()

150

In [8]:
from pyspark.sql.functions import col

df = df_iris.select(col("sepal length (cm)").alias("sepal_length"), col("sepal width (cm)").alias("sepal_width"), 
                 col("petal length (cm)").alias("petal_length"), col("petal width (cm)").alias("petal_width"), "Species")
df.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|Species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2|      0|
|         4.9|        3.0|         1.4|        0.2|      0|
|         4.7|        3.2|         1.3|        0.2|      0|
|         4.6|        3.1|         1.5|        0.2|      0|
|         5.0|        3.6|         1.4|        0.2|      0|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [9]:
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- Species: long (nullable = true)



In [10]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|      sepal_length|       sepal_width|      petal_length|       petal_width|           Species|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|               150|               150|               150|               150|               150|
|   mean| 5.843333333333334|3.0573333333333332|3.7580000000000005|1.1993333333333331|               1.0|
| stddev|0.8280661279778632|0.4358662849366984| 1.765298233259466|0.7622376689603464|0.8192319205190405|
|    min|               4.3|               2.0|               1.0|               0.1|                 0|
|    max|               7.9|               4.4|               6.9|               2.5|                 2|
+-------+------------------+------------------+------------------+------------------+------------------+



In [11]:
df.groupBy("Species").count().show()

+-------+-----+
|Species|count|
+-------+-----+
|      0|   50|
|      1|   50|
|      2|   50|
+-------+-----+



In [12]:
print(df.columns)

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'Species']


In [13]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=df.columns[:-1], outputCol="features")
data = assembler.transform(df)
data.show()

+------------+-----------+------------+-----------+-------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|Species|         features|
+------------+-----------+------------+-----------+-------+-----------------+
|         5.1|        3.5|         1.4|        0.2|      0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|      0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|      0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|      0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|      0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4|      0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3|      0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2|      0|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2|      0|[4.4,2.9,1.4,0.2]|
|         4.9|        3.1|         1.5|        0.1|      0|[4.9,

In [14]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kmeans = KMeans(featuresCol="features", k=3).fit(data)

result = kmeans.transform(data)
result.show(5)

+------------+-----------+------------+-----------+-------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|Species|         features|prediction|
+------------+-----------+------------+-----------+-------+-----------------+----------+
|         5.1|        3.5|         1.4|        0.2|      0|[5.1,3.5,1.4,0.2]|         1|
|         4.9|        3.0|         1.4|        0.2|      0|[4.9,3.0,1.4,0.2]|         1|
|         4.7|        3.2|         1.3|        0.2|      0|[4.7,3.2,1.3,0.2]|         1|
|         4.6|        3.1|         1.5|        0.2|      0|[4.6,3.1,1.5,0.2]|         1|
|         5.0|        3.6|         1.4|        0.2|      0|[5.0,3.6,1.4,0.2]|         1|
+------------+-----------+------------+-----------+-------+-----------------+----------+
only showing top 5 rows



In [15]:
## 비지도학습이므로 Label이 없으므로 어떤 비율로 clustering 되었는지 점검
result.groupBy("Species").count().show()
result.groupBy("prediction").count().show()

+-------+-----+
|Species|count|
+-------+-----+
|      0|   50|
|      1|   50|
|      2|   50|
+-------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   50|
|         2|   62|
|         0|   38|
+----------+-----+



### Cluster 시각화

In [16]:
centers = kmeans.clusterCenters()
centers

[array([6.85      , 3.07368421, 5.74210526, 2.07105263]),
 array([5.006, 3.428, 1.462, 0.246]),
 array([5.9016129 , 2.7483871 , 4.39354839, 1.43387097])]

In [ ]:
import matplotlib.pyplot as plt
for i in range(3):
    X = result.select("sepal_length", "sepal_width").where(f"prediction = {i}").toPandas().values
    plt.scatter(X[:, 0], X[:, 1], marker='.')
    plt.plot(centers[i][0], centers[i][1], marker='o', markeredgecolor='k', markersize=20)
  
plt.show()